## Get eps links

#### JJK

In [10]:
from bs4 import BeautifulSoup
import requests

html_content = "https://jujutsu-kaisen.fandom.com/wiki/List_of_Episodes"

# Parse the HTML content
response = requests.get(html_content)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables with the specified attributes
tables = soup.find_all('table', {
    'width': '100%',
    'style': 'border:1px #ED0000; overflow:hidden; text-align:center; align:center;',
    'border': '1'
})

# Initialize a list to hold all episodes
all_episodes = []

# Loop through each table and extract episode data
for table in tables:
    rows = table.find_all('tr')[1:]  # Skip the header row
    for row in rows:
        title_cell = row.find_all('td')[2]  # The third cell contains the title
        link = title_cell.find('a', href=True)  # Extract the <a> tag with a link
        if link:
            all_episodes.append({
                'name': link.text.strip(),
                'link': f"https://jujutsu-kaisen.fandom.com{link['href']}"  # Make the link absolute
            })

# Print extracted episodes
for episode in all_episodes:
    print(f"Episode: {episode['name']}, Link: {episode['link']}")

Episode: Ryomen Sukuna, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_1
Episode: For Myself, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_2
Episode: Girl of Steel, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_3
Episode: Fearsome Womb, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_4
Episode: Fearsome Womb, Part 2, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_5
Episode: After the Rain, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_6
Episode: Assault, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_7
Episode: Boring, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_8
Episode: Young Fish and Reverse Punishment, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_9
Episode: Idle Transfiguration, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_10
Episode: Narrow-Minded, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_11
Episode: To You Someday, Link: https://jujutsu-kaisen.fandom.com/wiki/Episode_12
Episode: Tomorrow, Link: htt

#### NARUTO

In [30]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage
html_content = "https://naruto.fandom.com/wiki/List_of_Animated_Media"

# Fetch and parse the HTML content
response = requests.get(html_content)
soup = BeautifulSoup(response.text, 'html.parser')

# Initialize dictionary to store data
data = {}

# Find all tables with their preceding headings
tables = soup.find_all('table', class_='box table coloured bordered innerbordered style-basic fill-horiz')

for table in tables:
    # Find the preceding headings
    sub_heading = table.find_previous('h3')  # Specific subcategory
    main_heading = table.find_previous('h2')  # Main category (e.g., Episodes, Movies)

    if sub_heading and main_heading:
        # Combine main and sub headings for unique categories like "Naruto: Original Movies"
        category_title = f"{main_heading.get_text(strip=True)} - {sub_heading.get_text(strip=True)}"
    elif main_heading:
        # Use the main heading if no subheading is found
        category_title = main_heading.get_text(strip=True)
    else:
        continue
    if category_title == "OVAs - Boruto: Naruto Next Generations":
        category_title = "OVAs"

    # Extract rows from the table
    rows = table.find_all('tr')[1:]  # Skip header row
    entries = []
    for idx, row in enumerate(rows, start=1):  # Use enumerate for dynamic numbering
        cells = row.find_all(['th', 'td'])
        if cells:
            if category_title == "OVAs":
                entry = {
                    "Number": idx,
                    "Title": cells[0].find('a').text.strip() if cells[0].find('a') else cells[0].text.strip(),
                    "Link": f"https://naruto.fandom.com{cells[0].find('a')['href']}" if cells[0].find('a') else None,
                }
            else:
                entry = {
                    "Number": cells[0].text.strip(),
                    "Title": cells[1].find('a').text.strip() if cells[1].find('a') else cells[1].text.strip(),
                    "Link": f"https://naruto.fandom.com{cells[1].find('a')['href']}" if cells[1].find('a') else None,
                }
            entries.append(entry)
    # Save entries under the category
    data[category_title] = entries

# Print results
for category, items in data.items():
    print(f"{category}")
    # for item in items:
    #     print(f"  Number: {item['Number']}")
    #     print(f"  Title: {item['Title']}")
    #     print(f"  Link: {item.get('Link')}\n")

Episodes - Naruto: Original
Episodes - Naruto: Shippūden
Episodes - Boruto: Naruto Next Generations
OVAs
Movies - Naruto: Original
Movies - Naruto: Shippūden


## Consolidece Eps

#### NO filler

In [ ]:
data
animefillerlist